In [1]:
from semanticnetworks.Concept_Network import *
from semanticnetworks.Analysis import *
import matplotlib.pyplot as plt
from matplotlib import pylab
from matplotlib import colors
from matplotlib.colors import LogNorm
import itertools
import powerlaw
from nltk.collocations import *
import traces
%matplotlib inline
%load_ext autoreload
%autoreload 2

Create all the networks and calculate the co-occurrence matrices and filtrations:

In [2]:
CN1 = Concept_Network()
t1 = CN1.cooc_for_text('Axler', 10)#, groups=Ci)#, groups=C)
CN2 = Concept_Network()
t2 = CN2.cooc_for_text('Edwards', 10)
CN3 = Concept_Network()
t3 = CN3.cooc_for_text('Lang', 8)
CN4 = Concept_Network()
t4 = CN4.cooc_for_text('Petersen', 5)
CN5 = Concept_Network()
t5 = CN5.cooc_for_text('Robbiano', 8)
CN6 = Concept_Network()
t6 = CN6.cooc_for_text('Bretscher', 9)
CN7 = Concept_Network()
t7 = CN7.cooc_for_text('Greub', 13)
CN8 = Concept_Network()
t8 = CN8.cooc_for_text('Hefferson', 5)
CN9 = Concept_Network()
t9 = CN9.cooc_for_text('Strang', 8)
CN0 = Concept_Network()
t0 = CN0.cooc_for_text('Wrong', 9)
texts = ['Wrong', 'Axler', 'Edwards', 'Lang', 'Petersen', 'Robbiano', 
         'Bretscher', 'Greub', 'Hefferson', 'Strang']

Yield percent: 1.0
Yield percent: 1.0



Yield percent: 1.0
Yield percent: 1.0



Yield percent: 1.0
Yield percent: 1.0



Yield percent: 1.0
Yield percent: 1.0



Yield percent: 1.0
Yield percent: 1.0



Yield percent: 1.0
Yield percent: 1.0



Yield percent: 0.9981851179673321
Yield percent: 1.0



Yield percent: 1.0
Yield percent: 1.0



Yield percent: 0.998898678414097
Yield percent: 1.0



Yield percent: 0.9982046678635548
Yield percent: 1.0


Let's save the texts themselves.

In [18]:
for i in range(10):
    print(texts[i])
    CN = globals()['CN{}'.format(i)]
    np.save('Textbooks/{}/clean_text.npy'.format(texts[i]), CN.text)

Wrong
Axler
Edwards
Lang
Petersen
Robbiano
Bretscher
Greub
Hefferson
Strang


First, we generate the actual sentence-granularity persistent homology for the true networks:

In [3]:
for i in range(10):
    print(texts[i])
    CN = globals()['CN{}'.format(i)]
    bars = CN.get_barcode(CN.dist_mat)
    np.save('Textbooks/{}/sentence_filtration.npy'.format(texts[i]),
            CN.dist_mat)
    np.save('Textbooks/{}/sentence_cooc.npy'.format(texts[i]),
            CN.cooc_mat)
    np.save('Textbooks/{}/sentence_bars.npy'.format(texts[i]), bars)

Wrong
Axler
Edwards
Lang
Petersen
Robbiano
Bretscher
Greub
Hefferson
Strang


Now we'll generate the continuous configuration null model graphs for each text, 100 for each

In [3]:
n_samples = 100
for i in range(10):
    print(texts[i])
    CN = globals()['CN{}'.format(i)]
    mats = []
    for _ in tqdm_notebook(range(n_samples)):
        mats.append(CN.cont_config_graph())
    mats = np.stack(mats)
    np.save('Textbooks/{}/cont_config_mats.npy'.format(texts[i]), mats)

Wrong


/usr/local/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:2381: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/usr/local/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1059: RuntimeWarning: invalid value encountered in subtract
  mu2 = mu2p - mu * mu
/usr/local/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1682: RuntimeWarning: divide by zero encountered in log
  return log(self._pdf(x, *args))
/usr/local/lib/python3.7/site-packages/scipy/stats/_continuous_distns.py:4339: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)


(3100.147902515249, 'burr', (1.7976081030760762, 1.1744637935831639, 0.058155345795563806, 0.5383912091401712))

Axler


(3707.5945943670085, 'burr', (1.8576443811198162, 1.0690103754024984, 0.0627033135023198, 0.5866859750796842))

Edwards


(1844.2440173452378, 'lognorm', (0.8024897898981113, 0.034947292211790305, 0.7079116637580649))

Lang


...

(3217.670517082068, 'fisk', (2.3199913772489893, 0.06427533873662954, 0.6945338864393606))

Bretscher


(5331.347437758494, 'burr', (1.7062521096704713, 1.3977728735455017, 0.03755174717739902, 0.46846553371535815))

Greub


(2961.815154414708, 'burr', (1.8730344455584496, 1.1164968260333428, 0.08311840455578037, 0.5544594020718934))

Hefferson


(5544.187048301326, 'burr', (1.9149670314849019, 1.162527726552161, 0.05807704403318088, 0.5663001911919197))

Strang


(6257.799783959007, 'burr', (1.8802491006302402, 1.3173617446319799, 0.06702246191278417, 0.4979895175538734))



Now we'll generate the random index network null models: filtration matrices and cooccurrence matrices, and corresponding oaat-persistent homology

In [7]:
for i in range(10):
    print(texts[i])
    CN = globals()['CN{}'.format(i)]
    cooc_mats = []
    filt_mats = []
    oaat_mats = []
    all_bars = []
    for _ in tqdm_notebook(range(n_samples)):
        t = CN.random_index_null()
        cooc_mats.append(t[0]) # add the graph matrix
        filt_mats.append(t[1]) # need the sentence-granularity filtration
        # matrix for doing stuff with core-peri and community development
        # add the oaat filtration matrix
        oaat_mat, cutoff, _ = CN.oaat_filtration(t[1])
        oaat_mats.append(oaat_mat)
        # calculate barcode
        bars = CN.get_barcode(oaat_mat) 
        all_bars.append((cutoff, bars))
    cooc_mats = np.stack(cooc_mats)
    filt_mats = np.stack(filt_mats)
    oaat_mats = np.stack(oaat_mats)
    np.save('Textbooks/{}/r_ind_cooc_mats.npy'.format(texts[i]), cooc_mats)
    np.save('Textbooks/{}/r_ind_filt_mats.npy'.format(texts[i]), filt_mats)
    np.save('Textbooks/{}/r_ind_oaat_mats.npy'.format(texts[i]), oaat_mats)
    np.save('Textbooks/{}/r_ind_bars.npy'.format(texts[i]), all_bars)

Wrong


Yield percent: 1.0


Yield percent: 1.0


Yield percent: 1.0


Yield percent: 1.0


...

Yield percent: 1.0


Yield percent: 1.0


Yield percent: 1.0


Yield percent: 1.0


Yield percent: 1.0


Now we generate the random sentence order null models: filtration matrices and cooccurrence matrices, and corresponding oaat-persistent homology

In [104]:
n_samples = 100
for i in range(10):
    print(texts[i])
    CN = globals()['CN{}'.format(i)]
    cooc_mats = []
    filt_mats = []
    oaat_mats = []
    all_bars = []
    all_sent_bars = []
    for _ in tqdm_notebook(range(n_samples)):
        t, _ = CN.rnd_sent_ord_null()
        cooc_mats.append(t[0]) # add the graph matrix
        filt_mats.append(t[1]) # need the sentence-granularity filtration
        # matrix for doing stuff with core-peri and community development
        # add the oaat filtration matrix
        oaat_mat, cutoff, _ = CN.oaat_filtration(t[1])
        oaat_mats.append(oaat_mat)
        # calculate barcode
        bars = CN.get_barcode(oaat_mat) 
        all_bars.append((cutoff, bars))
        bars = CN.get_barcode(t[1])
        all_sent_bars.append(bars)
    cooc_mats = np.stack(cooc_mats)
    filt_mats = np.stack(filt_mats)
    oaat_mats = np.stack(oaat_mats)
    np.save('Textbooks/{}/r_sentord_cooc_mats.npy'.format(texts[i]), cooc_mats)
    np.save('Textbooks/{}/r_sentord_filt_mats.npy'.format(texts[i]), filt_mats)
    np.save('Textbooks/{}/r_sentord_oaat_mats.npy'.format(texts[i]), oaat_mats)
    np.save('Textbooks/{}/r_sentord_bars.npy'.format(texts[i]), all_bars)
    np.save('Textbooks/{}/r_sentord_sent_bars.npy'.format(texts[i]), all_sent_bars)

Wrong


Axler


Edwards


Lang


Petersen


Robbiano


Bretscher


Greub


Hefferson


Strang


In [105]:
# also generate sentence-granularity homology for random index
n_samples = 100
r_ind_filts = load_r_ind_filt_mats()
for i in range(10):
    print(texts[i])
    CN = globals()['CN{}'.format(i)]
    all_sent_bars = []
    for j in tqdm_notebook(range(n_samples)):
        bars = CN.get_barcode(r_ind_filts[i][j])
        all_sent_bars.append(bars)
    np.save('Textbooks/{}/r_ind_sent_bars.npy'.format(texts[i]), all_sent_bars)

Wrong


Axler


Edwards


Lang


Petersen


Robbiano


Bretscher


Greub


Hefferson


Strang


Now we generate the oaat-persistent homology for the actual texts; we also do 100 each for this, since we're taking random ordering for within-sentence ordering:

In [8]:
for i in range(10):
    print(texts[i])
    CN = globals()['CN{}'.format(i)]
    filt_mats = []
    all_bars = []
    for _ in tqdm_notebook(range(n_samples)):
        # get the oaat filtration
        oaat_mat, cutoff, _ = CN.oaat_filtration(CN.dist_mat)
        filt_mats.append(oaat_mat)
        # calculate the barcode
        bars = CN.get_barcode(oaat_mat)
        all_bars.append((cutoff, bars))
    filt_mats = np.stack(filt_mats)
    np.save('Textbooks/{}/oaat_filt_mats.npy'.format(texts[i]), filt_mats)
    np.save('Textbooks/{}/oaat_bars.npy'.format(texts[i]), all_bars)

Wrong


Axler


Edwards


Lang


Petersen


Robbiano


Bretscher


Greub


Hefferson


Strang


Now we generate the node-ordered filtration null models for persistent homology.

In [9]:
for i in range(10):
    print(texts[i])
    CN = globals()['CN{}'.format(i)]
    all_bars = []
    for _ in tqdm_notebook(range(n_samples)):
        # get the node-ordered filtration
        mat, cutoff = CN.node_ordered_filtration()
        bars = CN.get_barcode(mat)
        all_bars.append((cutoff, bars))
    np.save('Textbooks/{}/node_ord_bars.npy'.format(texts[i]), all_bars)

Wrong


Axler


Edwards


Lang


Petersen


Robbiano


Bretscher


Greub


Hefferson


Strang


Now we generate the random edge filtration null models for persistent homology.

In [10]:
for i in range(10):
    print(texts[i])
    CN = globals()['CN{}'.format(i)]
    all_bars = []
    for _ in tqdm_notebook(range(n_samples)):
        # get the random edge filtration
        mat, cutoff, _ = CN.rnd_edge_filtration()
        bars = CN.get_barcode(mat)
        all_bars.append((cutoff, bars))
    np.save('Textbooks/{}/rand_edge_bars.npy'.format(texts[i]), all_bars)

Wrong


Axler


Edwards


Lang


Petersen


Robbiano


Bretscher


Greub


Hefferson


Strang


And finally, we generate the topological distance filtration null models for persistent homology (unused in paper).

In [11]:
for i in range(10):
    print(texts[i])
    CN = globals()['CN{}'.format(i)]
    all_bars = []
    for _ in tqdm_notebook(range(n_samples)):
        # get the random edge filtration
        mat, cutoff = CN.topo_dist_filtration()
        bars = CN.get_barcode(mat)
        all_bars.append((cutoff, bars))
    np.save('Textbooks/{}/topo_dist_bars.npy'.format(texts[i]), all_bars)

Wrong


Axler


Edwards


Lang


Petersen


Robbiano


Bretscher


Greub


Hefferson


Strang


We'll save all the index labels for each of the texts to file.

In [12]:
for i in range(10):
    print(texts[i])
    CN = globals()['CN{}'.format(i)]
    np.save('Textbooks/{}/index_labels.npy'.format(texts[i]), CN.index_labels)

Wrong
Axler
Edwards
Lang
Petersen
Robbiano
Bretscher
Greub
Hefferson
Strang


We'll also save the number of sentences of each text to file.

In [14]:
for i in range(10):
    print(texts[i])
    CN = globals()['CN{}'.format(i)]
    np.save('Textbooks/{}/n_sents.npy'.format(texts[i]), 
            len(CN.text))

Wrong
Axler
Edwards
Lang
Petersen
Robbiano
Bretscher
Greub
Hefferson
Strang


# END DATA GENERATION